In [2]:
import pandas as pd
from unidecode import unidecode
from re import sub

In [20]:
COGNOS_CATALOGUE_FILE_NAME = 'Cursos programados 2024_FINAL.xlsx'
COGNOS_CATALOGUE_SHEET_NAME = 'Hoja2'

VRAI_CATALOGUE_FILE_NAME = '3. Formulario preselección de cursos intercambio 2024-2. (1-13)_REZAGADOS.xlsx'
VRAI_CATALOGUE_SHEET_NAME = 'Catálogo VRAI'

GENERAL_COURSES_SHEET_NAME = 'Cursos generales'
SPORTS_COURSES_SHEET_NAME = 'Cursos deportivos'

columns_vrai_catalogue_df = {'nrc' : 'NRC',
                             'academic_unit' : 'Escuela',
                             'course_name' : 'Nombre del curso',
                             'sigla' : 'Sigla'}

columns_cognos_catalogue_df = {'nrc' : 'NRC',
                               'academic_unit' : 'Escuela',
                               'subject' : 'Materia',
                               'course_number' : 'Número Curso',
                               'course_name' : 'Nombre Curso'}

columns_students_choices_df = {'full_name' : 'Nombre completo',
                               'rut_uc' : 'RUT UC',
                               'course_name' : 'Nombre',
                               'sigla' : 'Sigla',
                               'nrc' : 'NRC'}

In [18]:
class DataFrameFormatter:
    def __init__(self, dataframe, columns:dict):
        self.dataframe = dataframe
        self.columns_dict = columns
        self.columns_list = list(columns.values())
        self.FORMATTED_COURSE_NAME_COLUMN_NAME = 'Nombre curso (formateado)'
        self.FORMATTED_COURSE_SIGLA_COLUMN_NAME = 'Sigla (formateado)'

    def strip_all_columns(self) -> None:
        self.dataframe[self.columns_list] = self.dataframe[self.columns_list].apply(lambda x : x.str.strip())
    
    def format_column(self, column_name, formatted_column_name) -> None:
        self.dataframe[formatted_column_name] = self.dataframe[column_name].apply(
            lambda x: TextFormatter(x).format_text()
        )

class CognosDataFrameFormatter(DataFrameFormatter):
    def __init__(self, dataframe, columns):
        super().__init__(dataframe, columns)

    def format_dataframe(self) -> None:
        self.strip_all_columns()
        self.format_course_number()
        self.create_sigla_column()
        self.format_column(self.columns_dict['course_name'], self.FORMATTED_COURSE_NAME_COLUMN_NAME)
        self.format_column('Sigla', self.FORMATTED_COURSE_SIGLA_COLUMN_NAME)

    def format_course_number(self) -> None:
        self.dataframe[self.columns_dict['course_number']] = self.dataframe[self.columns_dict['course_number']].apply(
            lambda x : CourseNumberFormatter(x).format_course_number()
        )

    def create_sigla_column(self) -> None:
        self.dataframe['Sigla'] = self.dataframe[self.columns_dict['subject']] + self.dataframe[self.columns_dict['course_number']]
        

class VraiDataFrameFormatter(DataFrameFormatter):
    def __init__(self, dataframe, columns):
        super().__init__(dataframe, columns)
        
    def format_dataframe(self) -> None:
        self.strip_all_columns()
        self.format_column(self.columns_dict['course_name'], self.FORMATTED_COURSE_NAME_COLUMN_NAME)
        self.format_column(self.columns_dict['sigla'], self.FORMATTED_COURSE_SIGLA_COLUMN_NAME)

class TextFormatter:
    def __init__(self, text:str):
        self.text = text

    def format_text(self) -> None:
        self.delete_accents()
        self.delete_special_characters()
        self.convert_to_lowercase()
        self.delete_spaces()
        return self.text

    def delete_accents(self) -> None:
        self.text = unidecode(str(self.text))

    def delete_special_characters(self) -> None:
        self.text = sub('[^a-zA-Z0-9]+', '', self.text)
    
    def convert_to_lowercase(self) -> None:
        self.text = self.text.lower()

    def delete_spaces(self) -> None:
        self.text = self.text.strip().replace(' ', '')
    
class CourseNumberFormatter:
    def __init__(self, course_number:str):
        self.course_number = course_number
    
    def format_course_number(self) -> str:
        if len(self.course_number) == 1:
            self.course_number = f'00{self.course_number}'
        
        elif len(self.course_number) == 2:
            self.course_number = f'0{self.course_number}'

        return self.course_number

In [14]:
df_cognos_catalogue = pd.read_excel(COGNOS_CATALOGUE_FILE_NAME, sheet_name=COGNOS_CATALOGUE_SHEET_NAME, dtype=str)[list(columns_cognos_catalogue_df.values())].astype(str)
df_cognos_catalogue_formatter = CognosDataFrameFormatter(df_cognos_catalogue, columns_cognos_catalogue_df)
df_cognos_catalogue_formatter.format_dataframe()
df_cognos_catalogue = df_cognos_catalogue_formatter.dataframe

In [15]:
df_vrai_catalogue = pd.read_excel(VRAI_CATALOGUE_FILE_NAME, sheet_name=VRAI_CATALOGUE_SHEET_NAME, dtype=str)[list(columns_vrai_catalogue_df.values())].astype(str)
df_vrai_catalogue_formatter = VraiDataFrameFormatter(df_vrai_catalogue, columns_vrai_catalogue_df)
df_vrai_catalogue_formatter.format_dataframe()
df_vrai_catalogue = df_vrai_catalogue_formatter.dataframe

In [21]:
df_general_courses = pd.read_excel(VRAI_CATALOGUE_FILE_NAME, sheet_name=GENERAL_COURSES_SHEET_NAME, dtype=str)[list(columns_students_choices_df.values())].astype(str)
df_general_courses_formatter = VraiDataFrameFormatter(df_general_courses, columns_students_choices_df)
df_general_courses_formatter.format_dataframe()
df_general_courses = df_general_courses_formatter.dataframe

In [22]:
df_sports_courses = pd.read_excel(VRAI_CATALOGUE_FILE_NAME, sheet_name=SPORTS_COURSES_SHEET_NAME, dtype=str)[list(columns_students_choices_df.values())].astype(str)
df_sports_courses_formatter = VraiDataFrameFormatter(df_sports_courses, columns_students_choices_df)
df_sports_courses_formatter.format_dataframe()
df_sports_courses = df_sports_courses_formatter.dataframe